# GCN for DAVIS 2016

In this notebook, a custom [PyTorch Geometric](https://rusty1s.github.io/pytorch_geometric/build/html/index.html) [InMemoryDataset](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/in_memory_dataset.html#InMemoryDataset) for the DAVIS 2016 dataset is created. The implementation is based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/create_dataset.html). The dataset is then used to train a simple GCN network as a first evaluation based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/introduction.html#learning-methods-on-graphs).

The dataset consists of single PyTorch Geometric [Data](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/data.html#Data) objects which model a single graph with various attributes. For this dataset, a graph for each contour is created. Hereby, each node of the graph represents one contour point. The feature of each node is the OSVOS feature vector from the next frame at this point. Each node is connected to its K nearest neighbours. The feature of each edge is the distance between the nodes it connects. The targets of each node is the translation it undergoes from the current to the next frame.

## Imports

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data.sampler import SequentialSampler

from torch_geometric.data import DataLoader

from pg_datasets.davis_2016 import DAVIS2016
from pg_networks.gcn import GCN
from pg_solvers.solver import Solver
from pg_utils.vis_utils import show_img_with_contour, plot_translation, show_loss

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Paths & Constants

In [2]:
PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH = 'pg_datasets/DAVIS_2016'#'pg_datasets/DAVIS_2016'
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
IMAGES_FOLDERS_PATH = 'DAVIS_2016/DAVIS/JPEGImages/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'

SKIP_SEQUENCES = []

SKIP_SEQUENCES = ['bmx-trees', 'bus', 'cows', 'dog-agility', 'horsejump-high', 
                  'horsejump-low', 'kite-walk', 'lucia', 'libby', 'motorbike',
                  'paragliding', 'rhino', 'scooter-gray', 'swing']

TRAIN_SEQUENCES = ['bmx-bumps', 'boat', 'breakdance-flare', 'bus', 
                   'car-turn', 'dance-jump', 'dog-agility', 'drift-turn', 
                   'elephant', 'flamingo', 'hike', 'hockey', 'horsejump-low', 
                   'kite-walk', 'lucia', 'mallard-fly', 'mallard-water', 
                   'motocross-bumps', 'motorbike', 'paragliding', 'rhino', 
                   'rollerblade', 'scooter-gray', 'soccerball', 'stroller',
                   'surf', 'swing', 'tennis', 'train']

VAL_SEQUENCES = ['blackswan', 'bmx-trees', 'breakdance', 'camel', 'car-roundabout',
                 'car-shadow', 'cows', 'dance-twirl', 'dog', 'drift-chicane', 
                 'drift-straight', 'goat', 'horsejump-high', 'kite-surf', 'libby', 
                 'motocross-jump', 'paragliding-launch', 'parkour', 'scooter-black', 
                 'soapbox']

BATCH_SIZE = 32
LAYER = 9
K = 32
EPOCHS_WO_AVEGRAD = 200

## Train and Val Dataset

In [3]:
train = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                  CONTOURS_FOLDERS_PATH, IMAGES_FOLDERS_PATH, TRANSLATIONS_FOLDERS_PATH, 
                  LAYER, K, EPOCHS_WO_AVEGRAD,
                  SKIP_SEQUENCES, TRAIN_SEQUENCES, VAL_SEQUENCES,
                  train=True)

Processing...
#1: bmx-bumps
['/home/christoph/in2364-adl4cv', '/opt/anaconda3/lib/python37.zip', '/opt/anaconda3/lib/python3.7', '/opt/anaconda3/lib/python3.7/lib-dynload', '', '/home/christoph/.local/lib/python3.7/site-packages', '/opt/anaconda3/lib/python3.7/site-packages', '/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/christoph/.ipython', '/home/christoph/in2364-adl4cv/OSVOS_PyTorch']
Start online training...
Constructing OSVOS architecture..
Initializing weights..
Done initializing train_seqs Dataset
Done initializing val_seqs Dataset
Start of Online Training, sequence: bmx-bumps
torch.Size([1, 3, 480, 854])
torch.Size([1, 1, 480, 854])
torch.Size([1, 3, 480, 854])
torch.Size([1, 1, 480, 854])
torch.Size([1, 3, 480, 854])
torch.Size([1, 1, 480, 854])
torch.Size([1, 3, 480, 854])
torch.Size([1, 1, 480, 854])
torch.Size([1, 3, 480, 854])
torch.Size([1, 1, 480, 854])
torch.Size([1, 3, 480, 854])
torch.Size([1, 1, 480, 854])
torch.Size([1, 3, 480, 854])
torch.

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [ ]:
val = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                CONTOURS_FOLDERS_PATH, IMAGES_FOLDERS_PATH, TRANSLATIONS_FOLDERS_PATH, 
                LAYER, K, EPOCHS_WO_AVEGRAD,
                SKIP_SEQUENCES, TRAIN_SEQUENCES, VAL_SEQUENCES,
                train=False)

In [ ]:
print("Train size: %i" % len(train))
print("Val size: %i" % len(val))

In [ ]:
num_to_display = 2

for i in range(num_to_display):
    
    # randomly select a sample
    rand_i = np.random.randint(0, len(train))
    sample = train[rand_i]
    show_img_with_contour(sample)

## Simple GCN

In [ ]:
model = GCN(in_channels=train[0].num_features, 
            out_channels=train[0].y.shape[1])

print(model)

### Overfit

In [ ]:
num_train = 80
num_val = 20

overfit_train_loader = DataLoader(train, batch_size=16, 
                                  shuffle=False, sampler=SequentialSampler(range(num_train)))
overfit_val_loader = DataLoader(train, batch_size=1, 
                                shuffle=False, sampler=SequentialSampler(range(num_val)))

# Load model and run the solver
overfit_model = GCN(in_channels=train[0].num_features, 
                    out_channels=train[0].y.shape[1])

overfit_solver = Solver(optim_args={"lr": 1e-4})
overfit_solver.train(overfit_model, overfit_train_loader, overfit_val_loader,
                     num_epochs=20, log_nth=100, verbose=True)

In [ ]:
show_loss(overfit_solver)

In [ ]:
model.cpu()
model.double()
model.eval()
num_to_display = 2

for i in range(num_to_display):
    
    rand_i = np.random.randint(0, num_val)
    print(rand_i)
    sample = train[rand_i]
    
    img = sample.img.detach().numpy().astype(np.int64)
    img = np.squeeze(img)
    img = np.moveaxis(img, 0, 2)
    
    translation = sample.y.detach().numpy()
    
    contour = sample.contour.detach().numpy()

    with torch.no_grad():
        pred_translation = model(sample).detach().numpy()

    translation_ground_truth = contour + translation
    translation_pred = contour + pred_translation
    
    plot_translation(img, translation_ground_truth, translation_pred)

### Train

In [ ]:
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)

# Load model and run the solver
model = GCN(in_channels=train[0].num_features, 
            out_channels=train[0].y.shape[1])

solver = Solver(optim_args={"lr": 1e-4})

solver.train(model, train_loader, val_loader,
             num_epochs=20, log_nth=5, verbose=True)

In [ ]:
show_loss(solver)

### Display trained outputs

In [ ]:
model.cpu()
num_to_display = 2
for i in range(num_to_display):
    
    rand_i = np.random.randint(0, len(val))
    sample = val[rand_i]
    
    img = sample.img.detach().numpy().astype(np.int64)
    img = np.squeeze(img)
    img = np.moveaxis(img, 0, 2)
    
    translation = sample.y.detach().numpy()
    
    contour = sample.contour.detach().numpy()

    with torch.no_grad():
        pred_translation = model(sample).detach().numpy()

    translation_ground_truth = contour + translation
    translation_pred = contour + pred_translation
    
    plot_translation(img, translation_ground_truth, translation_pred)